In [ ]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 16.7 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23919 sha256=7893bd6e872e119ccf74abe7cd8f920d74b926b8c916fe041126b9dadc78af61
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
data = yf.download("GOOGL" , start = "2018-01-01" , interval = '1d')

[*********************100%***********************]  1 of 1 completed


In [ ]:
data.shape

(893, 6)

In [ ]:
data.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,1053.020020,1075.979980,1053.020020,1073.209961,1073.209961,1588300
2018-01-03,1073.930054,1096.099976,1073.430054,1091.520020,1091.520020,1565900
2018-01-04,1097.089966,1104.079956,1094.260010,1095.760010,1095.760010,1302600


Understanding Trends with in the Data

In [ ]:
# Sort the data points based on indexes just for confirmation 
data.sort_index(inplace = True)

In [ ]:
# Remove any duplicate index 
data = data.loc[~data.index.duplicated(keep='first')]

In [ ]:
data.tail(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-07-16,2553.260010,2556.229980,2523.070068,2539.399902,2539.399902,1020600
2021-07-19,2521.790039,2524.100098,2470.100098,2491.560059,2491.560059,1398400
2021-07-20,2510.550049,2540.860107,2487.010010,2524.189941,2524.189941,1315600


In [ ]:
# Check for missing values 
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
# Get the statistics of the data
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,893.000000,893.000000,893.000000,893.000000,893.000000,8.930000e+02
mean,1405.943280,1420.839023,1391.615288,1406.873482,1406.873482,1.812910e+06
std,389.033451,391.866970,386.770193,389.654754,389.654754,8.426914e+05
min,984.320007,1012.119995,977.659973,984.669983,984.669983,4.656000e+05
25%,1132.160034,1143.780029,1117.199951,1130.699951,1130.699951,1.276000e+06
50%,1241.109985,1248.020020,1226.239990,1238.750000,1238.750000,1.585000e+06
75%,1516.910034,1536.969971,1505.449951,1518.630005,1518.630005,2.079200e+06
max,2572.979980,2586.530029,2558.290039,2564.739990,2564.739990,6.658900e+06


In [ ]:
import plotly.graph_objects as go

# Check the trend in Closing Values 
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Close'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Close')
fig.show()

In [ ]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Volume'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Volume')
fig.show()

Data Preparation

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
import pickle 
from tqdm.notebook import tnrange

In [ ]:
# Filter only required data 
data = data[['Close' , 'Volume']]
data.head(3)

,Close,Volume
Date,,
2018-01-02,1073.209961,1588300
2018-01-03,1091.520020,1565900
2018-01-04,1095.760010,1302600


In [ ]:
# Confirm the Testing Set length 
test_length = data[(data.index >= '2020-09-01')].shape[0]

In [ ]:
def CreateFeatures_and_Targets(data, feature_length):
    X = []
    Y = []

    for i in tnrange(len(data) - feature_length): 
        X.append(data.iloc[i : i + feature_length,:].values)
        Y.append(data["Close"].values[i+feature_length])

    X = np.array(X)
    Y = np.array(Y)

    return X , Y

In [ ]:
X , Y = CreateFeatures_and_Targets(data , 32)

In [ ]:
# Check the shapes
X.shape , Y.shape

((861, 32, 2), (861,))

In [ ]:
Xtrain , Xtest , Ytrain , Ytest = X[:-test_length] , X[-test_length:] , Y[:-test_length] , Y[-test_length:]

In [ ]:
# Check Training Dataset Shape 
Xtrain.shape , Ytrain.shape

((639, 32, 2), (639,))

In [ ]:
# Check Testing Dataset Shape
Xtest.shape , Ytest.shape

((222, 32, 2), (222,))

In [ ]:
# Create a Scaler to Scale Vectors with Multiple Dimensions 
class MultiDimensionScaler():
    def __init__(self):
        self.scalers = []

    def fit_transform(self , X):
        total_dims = X.shape[2]
        for i in range(total_dims):
            Scaler = MinMaxScaler()
            X[:, :, i] = Scaler.fit_transform(X[:, :, i])
            self.scalers.append(Scaler)
        return X

    def transform(self , X):
        for i in range(X.shape[2]):
            X[:, :, i] = self.scalers[i].transform(X[:,:,i])
        return X 

In [ ]:
Feature_Scaler = MultiDimensionScaler()
Xtrain = Feature_Scaler.fit_transform(Xtrain)
Xtest = Feature_Scaler.transform(Xtest)

In [ ]:
Target_Scaler = MinMaxScaler()
Ytrain = Target_Scaler.fit_transform(Ytrain.reshape(-1,1))
Ytest = Target_Scaler.transform(Ytest.reshape(-1,1))

In [ ]:
def save_object(obj , name : str):
    pickle_out = open(f"{name}.pck","wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

def load_object(name : str):
    pickle_in = open(f"{name}.pck","rb")
    data = pickle.load(pickle_in)
    return data

In [ ]:
# Save your objects for future purposes 
save_object(Feature_Scaler , "Feature_Scaler")
save_object(Target_Scaler , "Target_Scaler")

Model Building

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau

save_best = ModelCheckpoint("best_weights.h5", monitor='val_loss', save_best_only=True, save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,patience=4, min_lr=0.00001,verbose = 1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , Bidirectional

model = Sequential()

model.add(Bidirectional(LSTM(512 ,return_sequences=True , recurrent_dropout=0.0, input_shape=(32, 2))))
model.add(LSTM(256 ,recurrent_dropout=0.0))
model.add(Dropout(0.3))
model.add(Dense(64 , activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(32 , activation='elu'))
model.add(Dense(1 , activation='linear'))

In [ ]:
#optimizer = tf.keras.optimizers.Adam(learning_rate=0.002)
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.002)
model.compile(loss='mse', optimizer=optimizer)

In [ ]:
history = model.fit(Xtrain, Ytrain,
            epochs=10,
            batch_size = 1,
            verbose=1,
            shuffle=False ,
            validation_data=(Xtest , Ytest),
            callbacks=[reduce_lr , save_best])

Epoch 1/10
639/639 [==============================] - 34s 37ms/step - loss: 0.0074 - val_loss: 0.2995
Epoch 2/10
639/639 [==============================] - 22s 35ms/step - loss: 0.0094 - val_loss: 0.2526
Epoch 3/10
639/639 [==============================] - 23s 35ms/step - loss: 0.0076 - val_loss: 0.2576
Epoch 4/10
639/639 [==============================] - 23s 35ms/step - loss: 0.0063 - val_loss: 0.1928
Epoch 5/10
639/639 [==============================] - 22s 35ms/step - loss: 0.0059 - val_loss: 0.1559
Epoch 6/10
639/639 [==============================] - 23s 35ms/step - loss: 0.0056 - val_loss: 0.1607
Epoch 7/10
639/639 [==============================] - 23s 35ms/step - loss: 0.0050 - val_loss: 0.1688
Epoch 8/10
639/639 [==============================] - 22s 35ms/step - loss: 0.0050 - val_loss: 0.1305
Epoch 9/10
639/639 [==============================] - 23s 35ms/step - loss: 0.0048 - val_loss: 0.1234
Epoch 10/10
639/639 [==============================] - 22s 35ms/step - loss: 0.004

In [ ]:
# Checking the model Structure 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (1, 32, 1024)             2109440   
_________________________________________________________________
lstm_1 (LSTM)                (1, 256)                  1311744   
_________________________________________________________________
dropout (Dropout)            (1, 256)                  0         
_________________________________________________________________
dense (Dense)                (1, 64)                   16448     
_________________________________________________________________
dropout_1 (Dropout)          (1, 64)                   0         
_________________________________________________________________
dense_1 (Dense)              (1, 32)                   2080      
_________________________________________________________________
dense_2 (Dense)              (1, 1)                    3

In [ ]:
# Load the best weights
model.load_weights("best_weights.h5")

Visualize prediction on Test Set

In [ ]:
Predictions = model.predict(Xtest)

In [ ]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Ytest)

In [ ]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [ ]:
# Creating Sample Test Dataframe
test_dataframe_dict = {'Actual' : list(Actual) , 'Predicted' : list(Predictions)}
test_df = pd.DataFrame.from_dict(test_dataframe_dict)

test_df.index = data.index[-test_length:]

In [ ]:
test_df.head()

,Actual,Predicted
Date,,
2020-09-01,1655.079956,1617.119995
2020-09-02,1717.390015,1624.031372
2020-09-03,1629.510010,1630.291016
2020-09-04,1581.209961,1628.667358
2020-09-08,1523.599976,1624.444824


In [ ]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = test_df.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = test_df.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

Visualize Prediction on whole data

In [ ]:
Total_features = np.concatenate((Xtrain , Xtest) , axis = 0)

In [ ]:
Total_Targets = np.concatenate((Ytrain , Ytest) , axis = 0)

In [ ]:
Predictions = model.predict(Total_features)

In [ ]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Total_Targets)

In [ ]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [ ]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()